In [16]:
import pandas as pd
import pickle
import numpy as np
from tensorflow.keras.models import load_model

In [17]:
### Load the trained model, scaler pickle,onehot
model=load_model('model.h5')

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    lable_encoder_gender=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)


In [18]:

input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000.0,
    'NumOfProducts': 1,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000.0
}



In [19]:
geo_arr = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_cols = onehot_encoder_geo.get_feature_names_out(['Geography'])
geo_encoded_df = pd.DataFrame(geo_arr, columns=geo_cols)
geo_encoded_df

c:\Users\shukl\anaconda3\envs\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [20]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000.0,1,1,1,50000.0


In [21]:
##Encode Categorical variables
input_df['Gender'] = lable_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000.0,1,1,1,50000.0


In [22]:
##concatenation one hot encoded
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000.0,1,1,1,50000.0,1.0,0.0,0.0


In [23]:
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
        -0.91668767,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [24]:
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 103ms/step


array([[0.3105807]], dtype=float32)

In [25]:
prediction_proba=prediction[0][0]

In [26]:
prediction_proba

0.3105807

In [27]:
if prediction_proba >0.5:
    print('The Customer is likely to churn.')
else:
    print('The Customer is not likely to churn.')

The Customer is not likely to churn.
